In [ ]:
import numpy as np
import keras
import glob
import os
import random
import matplotlib.pyplot as plt
import cv2
import resnet
%matplotlib inline

Using TensorFlow backend.


ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)  

In [ ]:
## Generating data

img_width = 224
img_height = 224
img_channel = 3
batch_size = 40

cls = ['cats','dogs']
train_cls_label = {}
train_img_path = []

test_cls_label = {}
test_img_path = []


dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/training_set'

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    train_img_path = train_img_path + path
    train_cls_label.update({p:i for p in path})

print('train_img_path',len(train_img_path))
print('train_cls_label',len(train_cls_label))
    
dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/test_set'
path = []

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    test_img_path = test_img_path + path
    test_cls_label.update({p:i for p in path})

print('test_img_path',len(test_img_path))
print('test_cls_label',len(test_cls_label))

train_keys = list(train_cls_label.keys())
test_keys = list(test_cls_label.keys())

random.shuffle(train_keys)
random.shuffle(test_keys)

# To limit 
#train_keys = train_keys[:800]
#test_keys = test_keys[:800]

final_train = {}
final_test = {}

final_train.update({p:train_cls_label[p] for p in train_keys})
final_test.update({p:test_cls_label[p] for p in test_keys})



In [ ]:
print('final_train',len(final_train))
print('final_test',len(final_test))


In [ ]:
## Define Generator

def generator(dict_cls,batch_size):

  while True:
    
    start_position = 0
    end_position = batch_size

    dict_cls_key = list(dict_cls.keys())

    num_batches = len(dict_cls)//batch_size

    for batch in range(num_batches):
      batch_data = np.zeros((batch_size,img_height,img_width,3))
      batch_label = np.zeros((batch_size,2))

      
      dict_cls_batch_key = dict_cls_key[start_position:end_position]

      for index in range(batch_size):

        image = cv2.imread(dict_cls_batch_key[index])
        image = image[0:image.shape[0]-10,10:image.shape[1]-10]
        image = cv2.resize(image,(img_height,img_width))

        image = image/255
        #print(dict_cls_batch_key[index])
        batch_data[index,:,:,:] = image
        batch_label[index] = keras.utils.to_categorical(dict_cls[dict_cls_batch_key[index]],num_classes=2,dtype='int32')


      start_position = end_position
      end_position = end_position + batch_size
      #print('----------------------------------------------')
      yield batch_data , batch_label



In [ ]:
train_generator = generator(final_train,batch_size)
val_generator = generator(final_test,batch_size)

In [ ]:
import keras
from keras.models import Sequential , Model
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
num_classes = 2
num_epochs = 25

In [ ]:
model = resnet.ResnetBuilder.build_resnet_18((3,img_height,img_width),num_classes)

"""
model = Sequential()
model.add(Conv2D(32,(3, 3), padding='same',input_shape=(img_height,img_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(32))
#model.add(Activation('relu'))
#model.add(Dropout(0.45))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
"""

In [ ]:
# summary
#opt = optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
if (len(final_train)%batch_size) == 0:
    steps_per_epoch = int(len(final_train)/batch_size)
else:
    steps_per_epoch = (len(final_train)//batch_size) + 1

if (len(final_test)%batch_size) == 0:
    validation_steps = int(len(final_test)/batch_size)
else:
    validation_steps = (len(final_test)//batch_size) + 1

In [ ]:
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001) # write the REducelronplateau code here

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
  os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

callbacks_list = [checkpoint,LR]

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch,epochs=num_epochs,verbose=1, callbacks=callbacks_list,validation_data=val_generator,validation_steps=validation_steps)